In [ ]:
!pip install geopandas
!pip install geoplot

In [17]:
import os 
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import geopandas as gpd
import numpy as np
import glob
from datetime import datetime
import re
import folium
import branca.colormap as cm

In [18]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1i3TVrITc2827qOLH3SyxcPFPvLgAoB_7BXxMiKRW-SQ/edit#gid=0')
sheet = wb.worksheet('Sheet')
data = sheet.get_all_values()
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:]
df.head()

,num,date,regions,country,virus
1,1,18/03/2020,Djal,Kyrgystan,3
2,2,19/03/2020,Djal,Kyrgystan,3
3,3,20/03/2020,Djal,Kyrgystan,3
4,4,21/03/2020,Djal,Kyrgystan,5
5,5,22/03/2020,Djal,Kyrgystan,5


In [19]:
geoshape = os.path.join('drive/My Drive/Colab Notebooks', 'gadm36_KGZ_1.shp')

#Contain only one column which contains the polygons of KG regions in the order below
#Batken, Bishkek, Chui, DJAL, Naryn, Oshobl, Osh, Talas, Issyk
#I dont know why only one column 

regions = gpd.read_file(geoshape) 

#and we need add the column of regions in the same order
regions.insert(0, 
               column='regions', 
               value=['Batken', 'Bishkek', 'Chui', 'Djal', 'Naryn', 'Oshobl', 'Osh', 'Talas', 'Issyk'], 
               allow_duplicates=True)

#change the date to current day
# data = 'virus_' + '05042020.csv'
# virus_data_path = os.path.join('drive/My Drive/Colab Notebooks', data)
# virus_df = pd.read_csv(virus_data_path)
virus_df = df

#Join two dataframes: virus_df and regions dataframe
joined_df = virus_df.merge(regions, on='regions')


joined_df['time'] = pd.to_datetime(joined_df['date'], dayfirst=True).astype(int) / 10**9
joined_df['time'] = joined_df['time'].astype(int).astype(str)
joined_df['virus'] = joined_df['virus'].astype(int)

In [20]:
max_colour = joined_df['virus'].astype(int).max()
min_colour = joined_df['virus'].astype(int).min()
# print(max_colour, min_colour)
cmap = cm.linear.YlOrRd_09.scale(min_colour, max_colour).to_step(10)
#cmap = cm.linear.YlGn_09.scale(min_colour, max_colour)
#cmap = cm.linear.YlGnBu_09.scale(min_colour, max_colour).to_step(10)
joined_df['colour'] = joined_df['virus'].map(cmap)

In [21]:
#Create style_dictionary to time slider

country_list = joined_df['regions'].unique().tolist()
print(country_list)
country_idx = range(len(country_list))
print(country_idx)

style_dict = {}
for i in country_idx:
    country = country_list[i]
    result = joined_df[joined_df['regions'] == country]

    inner_dict = {}
    for _, r in result.iterrows():
      inner_dict[r['time']] = {'color': r['colour'], 'opacity': 0.7}
    style_dict[str(i)] = inner_dict


['Djal', 'Bishkek', 'Issyk', 'Osh', 'Naryn', 'Chui', 'Oshobl', 'Batken', 'Talas']
range(0, 9)


In [22]:
regions_df = joined_df[['geometry']]
regions_gdf = gpd.GeoDataFrame(regions_df)
regions_gdf = regions_gdf.drop_duplicates().reset_index()
regions_gdf
#Take only polygons of Bishkek and Osh

,index,geometry
0,0,"POLYGON ((73.81236 41.12698, 73.80660 41.12662..."
1,53,"POLYGON ((74.52780 42.84089, 74.51363 42.84553..."
2,106,"POLYGON ((79.45599 42.50289, 79.47148 42.49222..."
3,159,"POLYGON ((72.81273 40.56699, 72.81707 40.56406..."
4,212,"POLYGON ((74.81965 40.50986, 74.80692 40.50455..."
5,265,"POLYGON ((75.15929 42.84967, 75.16995 42.84834..."
6,318,"POLYGON ((71.96346 39.33260, 71.96275 39.33243..."
7,371,"POLYGON ((71.54883 39.70604, 71.55012 39.70386..."
8,424,"POLYGON ((72.14397 42.73814, 72.15188 42.74087..."


# Parametres of the map
# Regulate the image

*   height = '89%' not to scroll the map

*   and x, y coordinates are [41.3044, 74.7661] for centering

set the lontitude, latitude coordinates not to move to other countries

*   min_lat = 39.5
*   max_lat = 43.4
*   min_lon = 69
*   max_lon = 80

*   min_zoom = 5 to see neighbours



In [23]:
from folium.plugins import TimeSliderChoropleth

#change the coordinates to centrelize
slider_map = folium.Map(location=[41.3044, 74.7661], zoom_start=6.5, min_zoom=5, 
                        max_bounds=True, control_scale=False, 
                        tiles='OpenStreetMap',
                        height='88%', 
                        min_lat=39, max_lat=43.7, min_lon=69, max_lon=80,
                        )
#control scale -add the scale in the bottom
overlay = 'drive/My Drive/Colab Notebooks/jsons/KGZ.json'
_ = TimeSliderChoropleth(
    data=regions_gdf.to_json(),
    name='Spread',
    styledict=style_dict,
    overlay=True,

    ).add_to(slider_map)


#tooltip = 'Click me!'

# folium.Marker([42.8746, 74.5698], popup='<b>Bishkek</b>', icon=folium.Icon(color='green')).add_to(slider_map)
# folium.Marker([40.5140, 72.8161], popup='<b>Osh</b>', icon=folium.Icon(color='red')).add_to(slider_map)

geojson = 'drive/My Drive/Colab Notebooks/jsons/KGZ.json'
folium.GeoJson(geojson, name='geojson', 
               style_function=lambda feature: {
                   'fillColor': '#ebf0f0',
                   'color': 'black',
                   'weight': 2,
                   'dashArray': '5, 5',
                   'fillopacity' : 0.5,
                   'opacity' : 0.3
                   }).add_to(slider_map)




_ = cmap.add_to(slider_map)
cmap.caption = "Number of confirmed cases"
slider_map.save(outfile='Covid19_in_KG.html')   
# slider_map

In [ ]:
import os
import imageio

png_dir = 'drive/My Drive/Colab Notebooks/gif/'
images = []
for file_name in os.listdir(png_dir):
    print(file_name)
    if file_name.endswith('.png'):
        file_path = os.path.join(png_dir, file_name)
        images.append(imageio.imread(file_path))
imageio.mimsave('movie1.gif', images)

In [25]:
bichkek = [42.8746, 74.5698]
osh = [40.5140, 72.8161]

data = {'region' : ['Bishkek', 'Osh'], 'lat' : [42, 40], 'lon' : [74, 72], 'num' : ['14', '29']}
df = pd.DataFrame(data)
df

,region,lat,lon,num
0,Bishkek,42,74,14
1,Osh,40,72,29


In [28]:
from folium.plugins import MarkerCluster
mc = MarkerCluster()

for row in df.itertuples():
    mc.add_child(folium.Marker(location=[row.lat,  row.lon],
                               popup=row.region))

In [29]:
slider_map.add_child(mc)
slider_map